In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display, HTML

# Connect to the enriched metadata database
db_path = '../data/metazoa.db'
conn = sqlite3.connect(db_path)

# Load nodes
nodes_df = pd.read_sql_query(
    'SELECT ott_id, name, parent_ott_id FROM nodes',
    conn,
    dtype={'ott_id': str, 'parent_ott_id': str}
)

# Load metadata, including full_description and wiki_page_url if available
try:
    metadata_df = pd.read_sql_query(
        '''SELECT ott_id, common_name, description, full_description, image_url, wiki_page_url FROM metadata''',
        conn,
        dtype={'ott_id': str}
    )
    has_full = True
except Exception:
    metadata_df = pd.read_sql_query(
        '''SELECT ott_id, common_name, description, image_url, wiki_page_url FROM metadata''',
        conn,
        dtype={'ott_id': str}
    )
    has_full = False

# Merge only enriched taxa
combined = nodes_df.merge(metadata_df, on='ott_id', how='inner')

# Summary
display(HTML(f"<h4>Nodes with metadata: {len(combined)}/{len(nodes_df)} ({len(combined)/len(nodes_df):.2%})</h4>"))

# Only include taxa with a full description and an image
if has_full:
    filtered = combined.dropna(subset=['full_description', 'image_url'])
else:
    filtered = combined.dropna(subset=['description', 'image_url'])

# Prepare sample from filtered
display_count = min(10, len(filtered))
sample_df = filtered.sample(display_count).copy()

# Render inline images
def render_image(url):
    return f'<img src="{url}" width="80">' if pd.notna(url) else ''
sample_df['image'] = sample_df['image_url'].apply(render_image)

# After images, add Wikipedia link
sample_df['wikipedia_link'] = sample_df['wiki_page_url'].apply(
    lambda url: f'<a href="{url}" target="_blank">Wikipedia</a>' if pd.notna(url) else ''
)

# Define columns for display
cols = ['ott_id', 'name', 'common_name', 'description']
if has_full:
    cols.append('full_description')
cols.extend(['image', 'wikipedia_link'])

# Display as HTML table
display(HTML(sample_df[cols].to_html(escape=False, index=False)))

conn.close()


ott_id,name,common_name,description,full_description,image,wikipedia_link
357278,Ochlerotatus,Ochlerotatus,genus of insects,"Ochlerotatus is a genus of mosquito. Until 2000, it was ranked as a subgenus of Aedes but was reclassified as a distinct genus based on taxonomic studies. This change resulted in the renaming of many subgenus species, and revisions of related taxa in the Aedini tribe are ongoing. Some authors, however, still use traditional taxonomic names in their publications.",,Wikipedia
4142514,Erketu,Erketu,genus of reptiles (fossil),"Erketu (meaning ""Erketü Tengri"") is a genus of somphospondylan dinosaur that lived in Asia during the Late Cretaceous roughly between 96 million and 89 million years ago. It is known from the Bayan Shireh Formation of Mongolia and its fossils were between 2002 and 2003 during field expeditions. Erketu was first described in 2006 and later on in 2010 due to some cervicals that were left behind in the expedition. Erketu represents one of the first sauropods described from the Bayan Shireh Formation. The elongated cervical vertebrae of Erketu indicates that it was the sauropod with the longest neck relative to its body size.",,Wikipedia
4437007,Atomosia sayii,Atomosia sayii,species of insect,Atomosia sayii is a species of robber flies in the family Asilidae.,,Wikipedia
1044814,Amblyodipsas,Amblyodipsas,genus of reptiles,"Amblyodipsas is a genus of snakes found in Africa. Currently, 9 species are recognized. These snakes are often known as purple-glossed snakes or glossy snakes. Although rear-fanged, all species are considered harmless, but their venom has not been well studied. They should not be confused with the glossy snakes of the genus Arizona, which are found in North America.",,Wikipedia
5327518,Euscorpius avcii,Euscorpius avcii,species of arachnid,Euscorpius avcii is a species of scorpion in the family Euscorpiidae. It was described by Tropea et al. in 2012.,,Wikipedia
475746,Acanthodasys,Acanthodasys,genus of gastrotrichs,"Acanthodasys is a genus of worms belonging to the family Thaumastodermatidae. The species of this genus are found in Europe and the Americas. Species: Acanthodasys aculeatus Remane, 1927 Acanthodasys algarvense Hummon, 2008 Acanthodasys arcassonensis Kisielewski, 1987 Acanthodasys australis Bosco, Lourenço, Guidi, Balsamo, Hochberg & Garraffoni, 2020 Acanthodasys caribbeanensis Hochberg & Atherton, 2010 Acanthodasys carolinensis Hummon, 2008 Acanthodasys comtus Lee, 2012 Acanthodasys ericinus Lee, 2012 Acanthodasys fibrosus Clausen, 2004 Acanthodasys flabellicaudus Hummon & Todaro, 2009 Acanthodasys lineatus Clausen, 2000 Acanthodasys paurocactus Atherton & Hochberg, 2012 Acanthodasys silvulus Evans, 1992",,Wikipedia
4125193,Enyalius iheringii,Enyalius iheringii,species of reptile,"Enyalius iheringii, also known commonly as Ihering's fathead anole, is a species of lizard in the family Leiosauridae. The species is endemic to Brazil.",,Wikipedia
953090,Metapenaeopsis barbata,Metapenaeopsis barbata,species of crustacean,"Metapenaeopsis barbata, commonly known as the whiskered velvet shrimp, is a species of prawn in the family Penaeidae. It is benthic, living in the depth range of 20–70 m. M. barbata grows up to 108 mm in length.",,Wikipedia
888011,Cynoglossus,Cynoglossus,genus of tonguefish,"Cynoglossus is a genus of fish in the family Cynoglossidae. Most species are indigenous to the Indo-Pacific region, but there are also a few in warmer parts of the East Atlantic. They are commonly found in shallow waters on a muddy or sandy bottom, including estuaries and a few species are restricted to fresh water. One species Cynoglossus sinusarabici has invaded the Mediterranean Sea through the Suez Canal from the Red Sea, a process known as Lessepsian or Erythrean migration.",,Wikipedia
1070587,Glaucidium (genus in Opisthokonta),Glaucidium,genus of birds,"Pygmy owls are members of the genus Glaucidium. They belong to the typical owl family, Strigidae. The genus consists 

ott_id,name,common_name,description,full_description,image,wikipedia_link
357278,Ochlerotatus,Ochlerotatus,genus of insects,"Ochlerotatus is a genus of mosquito. Until 2000, it was ranked as a subgenus of Aedes but was reclassified as a distinct genus based on taxonomic studies. This change resulted in the renaming of many subgenus species, and revisions of related taxa in the Aedini tribe are ongoing. Some authors, however, still use traditional taxonomic names in their publications.",,Wikipedia
4142514,Erketu,Erketu,genus of reptiles (fossil),"Erketu (meaning ""Erketü Tengri"") is a genus of somphospondylan dinosaur that lived in Asia during the Late Cretaceous roughly between 96 million and 89 million years ago. It is known from the Bayan Shireh Formation of Mongolia and its fossils were between 2002 and 2003 during field expeditions. Erketu was first described in 2006 and later on in 2010 due to some cervicals that were left behind in the expedition. Erketu represents one of the first sauropods described from the Bayan Shireh Formation. The elongated cervical vertebrae of Erketu indicates that it was the sauropod with the longest neck relative to its body size.",,Wikipedia
4437007,Atomosia sayii,Atomosia sayii,species of insect,Atomosia sayii is a species of robber flies in the family Asilidae.,,Wikipedia
1044814,Amblyodipsas,Amblyodipsas,genus of reptiles,"Amblyodipsas is a genus of snakes found in Africa. Currently, 9 species are recognized. These snakes are often known as purple-glossed snakes or glossy snakes. Although rear-fanged, all species are considered harmless, but their venom has not been well studied. They should not be confused with the glossy snakes of the genus Arizona, which are found in North America.",,Wikipedia
5327518,Euscorpius avcii,Euscorpius avcii,species of arachnid,Euscorpius avcii is a species of scorpion in the family Euscorpiidae. It was described by Tropea et al. in 2012.,,Wikipedia
475746,Acanthodasys,Acanthodasys,genus of gastrotrichs,"Acanthodasys is a genus of worms belonging to the family Thaumastodermatidae. The species of this genus are found in Europe and the Americas. Species: Acanthodasys aculeatus Remane, 1927 Acanthodasys algarvense Hummon, 2008 Acanthodasys arcassonensis Kisielewski, 1987 Acanthodasys australis Bosco, Lourenço, Guidi, Balsamo, Hochberg & Garraffoni, 2020 Acanthodasys caribbeanensis Hochberg & Atherton, 2010 Acanthodasys carolinensis Hummon, 2008 Acanthodasys comtus Lee, 2012 Acanthodasys ericinus Lee, 2012 Acanthodasys fibrosus Clausen, 2004 Acanthodasys flabellicaudus Hummon & Todaro, 2009 Acanthodasys lineatus Clausen, 2000 Acanthodasys paurocactus Atherton & Hochberg, 2012 Acanthodasys silvulus Evans, 1992",,Wikipedia
4125193,Enyalius iheringii,Enyalius iheringii,species of reptile,"Enyalius iheringii, also known commonly as Ihering's fathead anole, is a species of lizard in the family Leiosauridae. The species is endemic to Brazil.",,Wikipedia
953090,Metapenaeopsis barbata,Metapenaeopsis barbata,species of crustacean,"Metapenaeopsis barbata, commonly known as the whiskered velvet shrimp, is a species of prawn in the family Penaeidae. It is benthic, living in the depth range of 20–70 m. M. barbata grows up to 108 mm in length.",,Wikipedia
888011,Cynoglossus,Cynoglossus,genus of tonguefish,"Cynoglossus is a genus of fish in the family Cynoglossidae. Most species are indigenous to the Indo-Pacific region, but there are also a few in warmer parts of the East Atlantic. They are commonly found in shallow waters on a muddy or sandy bottom, including estuaries and a few species are restricted to fresh water. One species Cynoglossus sinusarabici has invaded the Mediterranean Sea through the Suez Canal from the Red Sea, a process known as Lessepsian or Erythrean migration.",,Wikipedia
1070587,Glaucidium (genus in Opisthokonta),Glaucidium,genus of birds,"Pygmy owls are members of the genus Glaucidium. They belong to the typical owl family, Strigidae. The genus consists 